In [13]:
import numpy as np
import pandas as pd
import scipy as sp
import math

Проводим симуляцию монте-карло для вычисления критического значения критерия

In [20]:
def bhep_test_statistic(data, smoothing_parameter=1):
    n=data.shape[0]
    d=data.shape[1]
    if n <= d:
        print('not enough data!')
    S_n = np.cov(data, rowvar=False, bias=True)
    S_n_inverse_square_root = sp.linalg.sqrtm(np.linalg.inv(S_n))
    y_n=(data-np.mean(data, axis=0)) @ S_n_inverse_square_root
    jk_sum=0#считаем сумму по j,k из формулы тестовой статистики BHEP
    for j in range(n):
        for k in range(n):
            jk_sum+=np.exp(-(smoothing_parameter**2)*np.sum(np.square(data[j,:]-data[k,:]))/2)
    j_sum=0#считаем сумму по j из формулы тестовой статистики BHEP
    for j in range(n):
        j_sum+=np.exp(-(smoothing_parameter**2)*np.sum(np.square(data[j,:]))/(2*(1+smoothing_parameter**2)))
    bhep_test_statistic=(jk_sum/n) - (2*j_sum/((1+smoothing_parameter**2)**(d/2))) + n/((1+2*(smoothing_parameter**2))**(d/2))
    return bhep_test_statistic

In [58]:
def bhep_test(data, smoothing_parameter=1, sim_num=10000):
    n=data.shape[0]
    d=data.shape[1]
    d_cov = np.cov(data, rowvar=False, bias=True)
    d_mean = data.mean(axis=0)
    if n <= d:
        print('not enough data!')
    a=[]
    for i in range(sim_num):
        a.append(bhep_test_statistic(rng.multivariate_normal(mean=d_mean, cov=d_cov, size=n), smoothing_parameter=smoothing_parameter))
    bh_stat=bhep_test_statistic(data=data, smoothing_parameter=smoothing_parameter)
    distr_func=0
    for i in a:
        if bh_stat>i:
            distr_func+=1
        if bh_stat==i:
            print('for some reason two statistics are the same')
    distr_func/=sim_num
    return distr_func, bh_stat, a